# `labjack-controller` Demonstration Notebook
---
<h4><span style="color:gray">Montgomery, University of Southern Maine</span></h4>

## Notebook I: Implications of Configurations

The majority of operations included in the `labjack-controller` library are very straightforward and require no explaination. However, there are some configurations of the LJM devices that ultimately lead to non-intuitive behavior. In this notebook, we attempt to explain these results and explain the root cause behind it.

In [1]:
import pandas as pd
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show, output_notebook
output_notebook()

Loading BokehJS ...

This library is divided up into multiple objects; the two that we expect users to interact with is the `LabjackReader` object, which represents a LJM device (T4, T7, T7-Pro, and Digit devices in a very limited sense, and the `LJMLibrary` singleton, which controls global library functionality. It is very unlikely you will ever need to use `LJMLibrary` in most streaming applications.

In [2]:
from labjackcontroller.labtools import LabjackReader, LJMLibrary

### Step 1: Find a Labjack Device and Connect

Before doing anything, let's play dumb and assume we don't know what devices are plugged in, and list all devices that the computer knows about. To do this, we need a reference to the base library.

In [3]:
ljmlib_reference = LJMLibrary()

Now, we can list all connected devices. Output format is in the format (device model, connection type, serial number, IP) We'll use the zeroth device we find.

In [4]:
all_devices = ljmlib_reference.list_all()
all_devices

[('T7', 'USB', 470014713, '0.0.0.0')]

The default constructor for the `LabjackReader` expects you to give it at least a model name ("T7", for instance), and it will figure out the rest. This is great if you have a simple setup with only one or two devices, because the model is enough to serve as a unique identifier. In real life, this never happens, so you'll need to use the serial number or IP of the device as arguments to the `device_identifier` kwarg.

In [5]:
# Connect to the zeroth device, using the model, connection type, and serial number we found.
my_lj = LabjackReader(*all_devices[0][:3])

Before we proceed any further, it is important to understand that the method we connect to our LJM
device will affect latency when communicating to this notebook. WiFi (a T7 Pro only feature) has the
most latency, USB the second most, Ethernet has the least.

The effect of this latency is to increase the amount of time between the device recording data and the notebook recieving it; this means when you plot device recorded time vs. system recorded time, there will be a notable offset along one axis. Additionally, if you are performing a time-sensitive live analysis of the data recorded by your LJM device, knowledge of the presence of this latency is important.

While this latency varies on device, cable, firmware, and release of the `labjack-controller` library, we can get a general estimate on a Labjack T7 running firmware revision 1.046:

|**Method**         | **USB High-High** | **USB Other** | **Ethernet** | **WiFi** |
|-------------------|-------------------|---------------|--------------|----------|
| No I/O - Overhead | 0.6 	            | 2.2           | 1.1          | 6.7      |
| Read All DI       | 0.7               | 2.3           | 1.2          | 6.8      |
| Write All DO      | 0.7               | 2.3           | 1.2          | 6.8      |
| Write Both DACs   | 0.7               | 2.3           | 1.2          | 6.8      |

<span style="color:gray">Courtesy Labjack, <i>Command-Response Data Rates.</i> All times are in milliseconds.</span>


### Step 2: Streaming for a Labjack Device

Moving on to actual data recording, we provide the function `collect_data` for streaming. This function has the following outline and characteristics:
1. Connect to the LJM device, a process that takes about 1-2 seconds. `collect_data` will not try to open a connection if there is already one open, therefore, if you want to start a stream the moment an event occurs, you should pre-open a connection to the LJM device with the `open` function.
2. Start a stream, a process that takes 1-2 seconds and cannot be avoided.
2. Start the actual scan process by reading the values on all the specified channels and treating it like a row of data. This row is put into a data bundle of size \#`inputs` + 2 (for the device and recieved times) by `scans_per_read` and conveyed over the provided connection using as many packets as needed. `scans_per_read` is clearly constrained by the scan frequency; if your scan freqency is 100 Hz and you set `scans_per_read` to be 50 Hz, you will get two data bundles per second.
3. Scans record data of resolution `resolution`.

There's a lot to unpack here, so we'll start from the top.

#### Step 2a: Setting `resolution`

`resolution` changes the number of bits of precision that each data channel is recorded at. A higher resolution limits the maximum stream frequency, as each scan takes longer. This limitation varies by model, so the datatables are reproduced below.

<table><tr><td><img src='images/t7_data.PNG'></td><td><img src='images/t4_data.PNG'></td></tr></table>

In the `labjack-controller` package, the default resoultion is 4. Pick a setting that makes sense for your application.

#### Step 2b: Setting `scans_per_read`

The reason why `scans_per_read` can be set by the user is due to the fact that you can safely pick a value in the range \[1, frequency\] and get meaningfully different data characteristics.

When `scans_per_read` is small, each data row will have a on-device recorded time similar to the notebook's recorded recieving time, but you limit the frequency that can be effectively communicated at. When this parameter is similar to the frequency and `frequency` $\gg 1$, the host system gets a batch of many rows recorded at multiple times all at once, which leads to a System Time vs Device Time graph that is non-linear and instead looks like a collection of ascending 90-degree turns.

By default, `collect_data` configures this parameter to be as high as possible in order to max scanning at very high frequencies easy.

In [ ]:
# Scan the channel AIN0 in 10v mode for 15 seconds @ 10 kHz using the default scans_per_read
import time

def print_row(row):
    print(row)
    
def waste_time(row):
    time.sleep(1)

start = time.time_ns()
my_lj.collect_data(["AIN0"], [10.0], 30, 5000, resolution=1, callback_function=print_row, num_threads=16)
print("Actual time took: %f seconds" % ((time.time_ns() - start) / 1e9))

[10.104507446289062, 0.0, 0.41029858589172363]
[10.104507446289062, 0.0002, 0.4104897975921631]
[10.104507446289062, 0.0006, 0.41060495376586914][10.104507446289062, 0.0004, 0.41056060791015625][10.104507446289062, 0.0008, 0.41074371337890625][10.104507446289062, 0.0014, 0.41088414192199707][10.104507446289062, 0.001, 0.4107809066772461][10.104507446289062, 0.0012, 0.4108407497406006][10.104507446289062, 0.0016, 0.4109017848968506][10.104507446289062, 0.0022, 0.41100454330444336][10.104507446289062, 0.0028, 0.41110968589782715][10.104507446289062, 0.0026, 0.41109275817871094][10.104507446289062, 0.0024, 0.41106653213500977][10.104507446289062, 0.002, 0.4109632968902588][10.104507446289062, 0.0032, 0.41115903854370117][10.104507446289062, 0.003, 0.4111344814300537][10.104507446289062, 0.0018, 0.4109458923339844]
[10.104507446289062, 0.0034, 0.41119837760925293]






[10.104507446289062, 0.0038, 0.41126227378845215]

[10.104507446289062, 0.0036, 0.4112284183502197]
[10.104507446289062, 

[10.104507446289062, 0.0332, 0.4177265167236328][10.104507446289062, 0.033, 0.4177095890045166]

[10.104507446289062, 0.0334, 0.41776299476623535][10.104507446289062, 0.034, 0.4178900718688965]

[10.104507446289062, 0.0336, 0.41779494285583496][10.104507446289062, 0.0338, 0.417830228805542]




[10.104507446289062, 0.0344, 0.4179956912994385][10.104507446289062, 0.0348, 0.4180417060852051]
[10.104507446289062, 0.035, 0.4180755615234375][10.104507446289062, 0.0342, 0.41794633865356445]

[10.104507446289062, 0.0352, 0.4180924892425537][10.104507446289062, 0.0354, 0.4181089401245117]

[10.104507446289062, 0.036, 0.418210506439209][10.104507446289062, 0.0358, 0.41816210746765137]
[10.104507446289062, 0.0346, 0.41802120208740234]
[10.104507446289062, 0.0364, 0.41826558113098145]


[10.104507446289062, 0.0366, 0.41828298568725586][10.104507446289062, 0.0362, 0.4182403087615967]
[10.104507446289062, 0.0356, 0.4181375503540039]
[10.104507446289062, 0.0372, 0.41848134994506836]

[10.10450744628

[10.104507446289062, 0.066, 0.42287683486938477]
[10.104507446289062, 0.0668, 0.42299962043762207][10.104507446289062, 0.0664, 0.42291951179504395]
[10.104507446289062, 0.0666, 0.4229552745819092]
[10.104507446289062, 0.0672, 0.42305922508239746][10.104507446289062, 0.0674, 0.4230811595916748][10.104507446289062, 0.067, 0.42302656173706055][10.104507446289062, 0.0676, 0.4230985641479492]

[10.104507446289062, 0.068, 0.4232149124145508]
[10.104507446289062, 0.0678, 0.42316579818725586]


[10.104507446289062, 0.0684, 0.42333126068115234]
[10.104507446289062, 0.0686, 0.4233977794647217][10.104507446289062, 0.069, 0.4234335422515869]

[10.104507446289062, 0.0692, 0.4234507083892822][10.104507446289062, 0.0694, 0.4234910011291504]


[10.104507446289062, 0.0688, 0.4234166145324707][10.104507446289062, 0.0682, 0.4232473373413086]
[10.104507446289062, 0.0702, 0.4235725402832031]
[10.104507446289062, 0.0696, 0.42351198196411133][10.104507446289062, 0.0708, 0.42366814613342285][10.10450744628906

[10.104507446289062, 0.0984, 0.4406874179840088]
[10.104507446289062, 0.0998, 0.44077587127685547][10.104507446289062, 0.1, 0.44080686569213867]
[10.104507446289062, 0.1006, 0.4408419132232666][10.104507446289062, 0.1002, 0.4408226013183594][10.104507446289062, 0.0988, 0.44072580337524414]
[10.104507446289062, 0.1004, 0.44083261489868164][10.104507446289062, 0.1008, 0.4408707618713379]




[10.104507446289062, 0.101, 0.440889835357666]



[10.104507446289062, 0.1018, 0.4409606456756592][10.104507446289062, 0.1016, 0.4409494400024414]
[10.104507446289062, 0.1012, 0.44090819358825684][10.104507446289062, 0.1024, 0.4410109519958496][10.104507446289062, 0.1014, 0.44093775749206543][10.104507446289062, 0.1022, 0.44100069999694824]
[10.104507446289062, 0.102, 0.4409787654876709]
[10.104507446289062, 0.1026, 0.4410278797149658][10.104507446289062, 0.1028, 0.4410579204559326][10.104507446289062, 0.1034, 0.44112372398376465]
[10.104507446289062, 0.1036, 0.4412574768066406][10.104507446289062, 0

[10.104507446289062, 0.1326, 0.44458603858947754][10.104507446289062, 0.1332, 0.4446430206298828][10.104507446289062, 0.133, 0.444627046585083]
[10.104507446289062, 0.1334, 0.4446578025817871]


[10.104507446289062, 0.1336, 0.444683313369751]

[10.104507446289062, 0.134, 0.4447143077850342]
[10.104507446289062, 0.1338, 0.4447009563446045]
[10.104507446289062, 0.1342, 0.44473838806152344]
[10.104507446289062, 0.1344, 0.4447638988494873]
[10.104507446289062, 0.135, 0.4448122978210449]

[10.104507446289062, 0.1312, 0.4444427490234375]


[10.104507446289062, 0.1346, 0.44478368759155273][10.104507446289062, 0.1354, 0.4449026584625244][10.104507446289062, 0.1352, 0.44483280181884766][10.104507446289062, 0.1356, 0.4449191093444824][10.104507446289062, 0.1364, 0.445009708404541][10.104507446289062, 0.1362, 0.4449939727783203][10.104507446289062, 0.1348, 0.44479846954345703][10.104507446289062, 0.1358, 0.44494128227233887][10.104507446289062, 0.1366, 0.44507694244384766]

[10.104507446289062, 0

[10.104507446289062, 0.1658, 0.4490211009979248]


[10.104507446289062, 0.1662, 0.4490644931793213][10.104507446289062, 0.166, 0.4490518569946289]
[10.104507446289062, 0.1664, 0.44907641410827637]


[10.104507446289062, 0.1676, 0.44916439056396484][10.104507446289062, 0.167, 0.4491150379180908][10.104507446289062, 0.1678, 0.44917845726013184][10.104507446289062, 0.1666, 0.44909191131591797]
[10.104507446289062, 0.168, 0.44919657707214355]
[10.104507446289062, 0.1686, 0.4492361545562744][10.104507446289062, 0.1672, 0.44913315773010254]



[10.104507446289062, 0.1668, 0.44910335540771484][10.104507446289062, 0.1682, 0.44920897483825684]
[10.104507446289062, 0.1684, 0.44922351837158203]
[10.104507446289062, 0.1674, 0.44915199279785156]

[10.104507446289062, 0.1688, 0.44925475120544434]
[10.104507446289062, 0.1692, 0.4492802619934082]


[10.104507446289062, 0.1694, 0.4492950439453125][10.104507446289062, 0.169, 0.4492676258087158]
[10.104507446289062, 0.1704, 0.45043301582336426][10.104507

[10.104507446289062, 0.198, 0.455336332321167][10.104507446289062, 0.199, 0.45541906356811523]
[10.104507446289062, 0.1994, 0.4554405212402344]




[10.104507446289062, 0.1998, 0.4554612636566162]

[10.104507446289062, 0.2, 0.45547056198120117][10.104507446289062, 0.1996, 0.45545220375061035]

In [ ]:
fig_width = 800
tools = ["box_select", "box_zoom", "hover", "reset"]

In [ ]:
# Note the effect that waiting to assemble large data transmissions has.
datarun_source = ColumnDataSource(my_lj.to_dataframe()[4:])
time_fig = figure(plot_width=fig_width, title="Time vs. System Time",
                  x_axis_label="Device Time (sec)", y_axis_label="System Time (Sec)", tools=tools)
time_fig.line(source=datarun_source, x="Time", y="System Time")

show(time_fig)

Now, let's demonstrate the effect of a small `scans_per_read`:

In [ ]:
# Scan the channel AIN0 in 10v mode for 15 seconds @ 10 kHz using the default scans_per_read
my_lj.collect_data(["AIN0"], [10.0], 15, 10000, resolution=1, scans_per_read=1)

In [ ]:
# Our graph looks like a 45-degree line with a vertical offset to account to latency, as expected.
datarun_source = ColumnDataSource(my_lj.to_dataframe()[4:])
time_fig = figure(plot_width=fig_width, title="Time vs. System Time",
                  x_axis_label="Device Time (sec)", y_axis_label="System Time (Sec)", tools=tools)
time_fig.line(source=datarun_source, x="Time", y="System Time")

show(time_fig)

In [ ]:
from multiprocessing import Pool

In [ ]:
help(Pool)

In [ ]:
pool = Pool(processes=4)

In [ ]:
help(pool)